In [1]:
import os

emotions = ['happy', 'sad', 'hungry']
base_dir = 'dataset'

for emotion in emotions:
    os.makedirs(os.path.join(base_dir, emotion), exist_ok=True)

In [2]:
import os
import csv

base_dir = 'dataset'
csv_file = 'annotation.csv'

with open(csv_file, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['animal_images', 'animal_emotions'])
    
    for emotion in emotions:
        emotion_dir = os.path.join(base_dir, emotion)
        for image_name in os.listdir(emotion_dir):
            if image_name.endswith(('.png', '.jpg', '.jpeg')):
                image_path = os.path.join(emotion_dir, image_name)
                writer.writerow([image_path, emotion])

In [10]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import cv2
import os

In [11]:
# Load the CSV file
data = pd.read_csv('annotation.csv')

# Display the first few rows
data.head()

,animal_images,animal_emotions
0,dataset\happy\Cat (2).jpg,happy
1,dataset\happy\Cat (3).jpg,happy
2,dataset\happy\Cat (4).jpg,happy
3,dataset\happy\Cat (5).jpg,happy
4,dataset\happy\Cat (6).jpg,happy


In [12]:
# Initialize the image data and labels
images = []
labels = []

# Define the image size
IMG_SIZE = 128

# Iterate over the dataset
for idx, row in data.iterrows():
    image_path = row['animal_images']
    label = row['animal_emotions']
    
    # Load the image
    image = load_img(image_path, target_size=(IMG_SIZE, IMG_SIZE))
    image = img_to_array(image)
    image = image / 255.0  # Normalize the image
    
    images.append(image)
    labels.append(label)

# Convert lists to numpy arrays
images = np.array(images)
labels = np.array(labels)


In [13]:
# Encode the labels
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)


In [14]:
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

In [22]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(3, activation='softmax')  # Assuming three classes: happy, sad, hungry
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.summary()


C:\Users\Dell\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 126, 126, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 63, 63, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 61, 61, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 30, 30, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 28, 28, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 14, 14, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_5 (Flatten)                  │ (None, 25088)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 128)                 │       3,211,392 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ (None, 3)                   │             387 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,305,027 (12.61 MB)

 Trainable params: 3,305,027 (12.61 MB)

 Non-trainable params: 0 (0.00 B)

In [26]:
history = model.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test))

Epoch 1/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 178ms/step - accuracy: 1.0000 - loss: 4.6689e-06 - val_accuracy: 0.2143 - val_loss: 6.0623
Epoch 2/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 147ms/step - accuracy: 1.0000 - loss: 4.6440e-06 - val_accuracy: 0.2143 - val_loss: 6.0680
Epoch 3/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 151ms/step - accuracy: 1.0000 - loss: 4.6293e-06 - val_accuracy: 0.2143 - val_loss: 6.0742
Epoch 4/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 149ms/step - accuracy: 1.0000 - loss: 4.6113e-06 - val_accuracy: 0.2143 - val_loss: 6.0790
Epoch 5/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 147ms/step - accuracy: 1.0000 - loss: 4.5497e-06 - val_accuracy: 0.2143 - val_loss: 6.0834
Epoch 6/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 149ms/step - accuracy: 1.0000 - loss: 4.0581e-06 - val_accuracy: 0.2143 - val_loss: 6.0884
Epoch 7/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 145ms/step - accuracy: 1.0000 - loss: 3.9213e-06 - val_accuracy: 0.2143 - val_loss: 6.0937
Epoch 8/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 145ms/step - accuracy: 1.0000 - loss: 4.174

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 158ms/step - accuracy: 1.0000 - loss: 2.9726e-06 - val_accuracy: 0.2143 - val_loss: 6.3340
Epoch 51/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 150ms/step - accuracy: 1.0000 - loss: 3.1851e-06 - val_accuracy: 0.2143 - val_loss: 6.3388
Epoch 52/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 156ms/step - accuracy: 1.0000 - loss: 3.0744e-06 - val_accuracy: 0.2143 - val_loss: 6.3440
Epoch 53/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 145ms/step - accuracy: 1.0000 - loss: 2.7249e-06 - val_accuracy: 0.2143 - val_loss: 6.3489
Epoch 54/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 164ms/step - accuracy: 1.0000 - loss: 2.7062e-06 - val_accuracy: 0.2143 - val_loss: 6.3543
Epoch 55/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 162ms/step - accuracy: 1.0000 - loss: 3.2685e-06 - val_accuracy: 0.2143 - val_loss: 6.3589
Epoch 56/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 143ms/step - accuracy: 1.0000 - loss: 2.8421e-06 - val_accuracy: 0.2143 - val_loss: 6.3635
Epoch 57/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 157ms/step - accuracy: 1.0000 - loss: 3.2448e-06

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 151ms/step - accuracy: 1.0000 - loss: 2.2408e-06 - val_accuracy: 0.2143 - val_loss: 6.5748
Epoch 100/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 171ms/step - accuracy: 1.0000 - loss: 2.1510e-06 - val_accuracy: 0.2143 - val_loss: 6.5804


In [9]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {accuracy*100:.2f}%')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - accuracy: 0.2857 - loss: 1.7885
Test Accuracy: 28.57%


In [24]:
model.save('animal_emotion_model.h5')

In [25]:
def predict_emotion(image_path):
    # Load the image
    image = load_img(image_path, target_size=(IMG_SIZE, IMG_SIZE))
    image = img_to_array(image)
    image = image / 255.0  # Normalize the image
    image = np.expand_dims(image, axis=0)
    
    # Predict the emotion
    prediction = model.predict(image)
    emotion = label_encoder.inverse_transform([np.argmax(prediction)])[0]
    
    return emotion

# Example usage
test_image_path = 'dataset\happy\Monkey (2).jpg'
emotion = predict_emotion(test_image_path)
print(f'The predicted emotion is: {emotion}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
The predicted emotion is: happy


In [27]:
def predict_emotions(image_paths):
    emotions = []
    for image_path in image_paths:
        emotion = predict_emotion(image_path)
        emotions.append((os.path.basename(image_path), emotion))
    
    return emotions

# Example usage with a list of image paths
test_image_paths = ['dataset\happy\Monkey (2).jpg', 'dataset\sad\Cat (2).jpg']
emotions = predict_emotions(test_image_paths)
for animal, emotion in emotions:
    print(f'{animal}: {emotion}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Monkey (2).jpg: happy
Cat (2).jpg: sad


In [ ]:
import os

emotions = ['happy', 'sad', 'hungry']
base_dir = 'dataset'

for emotion in emotions:
    os.makedirs(os.path.join(base_dir, emotion), exist_ok=True)
    
import os
import csv

base_dir = 'dataset'
csv_file = 'annotation.csv'

with open(csv_file, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['animal_images', 'animal_emotions'])
    
    for emotion in emotions:
        emotion_dir = os.path.join(base_dir, emotion)
        for image_name in os.listdir(emotion_dir):
            if image_name.endswith(('.png', '.jpg', '.jpeg')):
                image_path = os.path.join(emotion_dir, image_name)
                writer.writerow([image_path, emotion])
                
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import cv2
import os

# Load the CSV file
data = pd.read_csv('annotation.csv')

# Display the first few rows
data.head()

# Initialize the image data and labels
images = []
labels = []

# Define the image size
IMG_SIZE = 128

# Iterate over the dataset
for idx, row in data.iterrows():
    image_path = row['animal_images']
    label = row['animal_emotions']
    
    # Load the image
    image = load_img(image_path, target_size=(IMG_SIZE, IMG_SIZE))
    image = img_to_array(image)
    image = image / 255.0  # Normalize the image
    
    images.append(image)
    labels.append(label)

# Convert lists to numpy arrays
images = np.array(images)
labels = np.array(labels)

# Encode the labels
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)

X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(3, activation='softmax')  # Assuming three classes: happy, sad, hungry
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.summary()

history = model.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test))

def predict_emotion(image_path):
    # Load the image
    image = load_img(image_path, target_size=(IMG_SIZE, IMG_SIZE))
    image = img_to_array(image)
    image = image / 255.0  # Normalize the image
    image = np.expand_dims(image, axis=0)
    
    # Predict the emotion
    prediction = model.predict(image)
    emotion = label_encoder.inverse_transform([np.argmax(prediction)])[0]
    
    return emotion

# Example usage
test_image_path = 'dataset\happy\Monkey (2).jpg'
emotion = predict_emotion(test_image_path)
print(f'The predicted emotion is: {emotion}')

def predict_emotions(image_paths):
    emotions = []
    for image_path in image_paths:
        emotion = predict_emotion(image_path)
        emotions.append((os.path.basename(image_path), emotion))
    
    return emotions

# Example usage with a list of image paths
test_image_paths = ['dataset\happy\Monkey (2).jpg', 'dataset\sad\Cat (2).jpg']
emotions = predict_emotions(test_image_paths)
for animal, emotion in emotions:
    print(f'{animal}: {emotion}')